* 전처리 할 때 read_csv, to_csv 시도하는 경우
* encoding="CP949" 인코딩 옵션 넣어줘야 한글 안 깨집니다
* read_csv에 CP949 안 먹히면 UTF8로 바꿔주세요

In [1]:
# 전처리를 위한 라이브러리 불러오기
import pandas as pd
import numpy as np

In [2]:
# GDP는 아주 단순합니다
# transpose()로 컬럼 세로로 만든 다음
# 필요한 부분만 iloc 이용해서 추출해서 내보내면 끝

gdp = pd.read_csv("../raw_data/국내총생산.csv", encoding="UTF8").transpose()
gdp_real = gdp.iloc[4:].rename(columns={0:"GDP"})
gdp_real.to_csv("../전처리/GDP.csv", encoding="CP949")

In [3]:
# 소비자물가지수 데이터도 단순합니다
# transpose()로 컬럼 세로로 만들어서
# 필요한 부분을 추출하고 내보내면 끝

sobija = pd.read_csv("../raw_data/소비자물가지수.csv").transpose()
sobija_real = sobija.iloc[5:].rename(columns={0:"소비자물가지수"})
sobija_real.to_csv("../전처리/소비자물가지수.csv", encoding="CP949")

In [4]:
# 주택종합 매매가격지수, 전세가격지수 전처리

# 매매가격지수 데이터 가져오기
data = pd.read_excel("../raw_data/주택종합 매매가격지수_20220808.xlsx")

# 데이터 확인 후 필요한 부분만 추출해 회전하고 합치기
data_s = data[0:1].transpose().reset_index().drop(index=0)

# 데이터 확인 (필요없으면 주석처리)
# data_s

In [5]:
# 주택지수에 대한 시계열 처리 수행
data_s['index'] = pd.to_datetime(data_s['index'], format="%Y-%m-%d %HH:%M:%S").dt.strftime("%Y-%m")

In [6]:
# CSV로 뽑아줍시다
# 컬럼 이름 바꿔주는 rename에 이름 새로 지정하세요
data_s.rename(columns={0:"주택종합 매매가격지수"}).reset_index(drop=True).to_csv("../전처리/주택종합_매매가격지수.csv",encoding='CP949')

In [7]:
# 전세가격 지수는 똑같으니까 한번에 갑니다

data2 = pd.read_excel("../raw_data/주택종합 전세가격지수_20220808.xlsx")
data2_s = data2[0:1].transpose().reset_index().drop(index=0)
data2_s['index'] = pd.to_datetime(data2_s['index'], format="%Y-%m-%d %HH:%M:%S").dt.strftime("%Y-%m")
data2_s.rename(columns={0:"주택종합 전세가격지수"}).reset_index(drop=True).to_csv("../전처리/주택종합_전세가격지수.csv",encoding='CP949')

In [8]:
# 실업자 갑니다
job = pd.read_csv("../raw_data/연령별_경제활동인구_총괄_20220808152350(월별).csv", encoding='CP949').transpose()

# 임시로 데이터를 담아놓을 데이터프레임과 리스트 선언
job_test = pd.DataFrame()
job_test_real = []

# 반복문으로 돌릴건데
# 가져온 csv 파일 끝까지 돌릴 거고
for i in range(len(job)):
    # 맨 끝에 도달하면 탈출합니다
    if(i*8-4 == len(job)+3): break
    # 맨 처음 한 번은 건너 뜁니다 (우리가 원하는 데이터 값이 아니라서)
    if(i==0): continue

    # 실업률만 골라서 저장하고 있습니다
    job_test = job.iloc[i*8-4]
    # 실업률을 리스트에 저장하고 있습니다
    job_test_real.append(job_test)

# 불필요한 컬럼을 삭제하고, 컬럼의 이름을 바꿔서 내보내봅시다
job_test_real_df = pd.DataFrame(job_test_real).drop(columns=[0]).rename(columns={1:"15세 이상", 2:"15-19", 3:"20-29", 4:"30-39", 5:"40-49", 6:"50-59", 7:"60세 이상", 8:"15-64", 9:"15-24", 10:"15-29"})

# 실업자로 구하고 싶으면 i*8-4, len(job)+3 으로 변경해주세요
# 실업률로 구하고 싶으면 i*8-1, len(job)+6니으로 변경해주세요
# iloc로 필요한 성분을 조회해서 그 때 그 때 위치에 맞춰서 조정하면 다른 데이터도 뽑을 수 있을 거예요

In [9]:
# 실업자 index에 대한 시계열 처리 후 저장까지 갑시다

job_test_real_df = job_test_real_df.reset_index()
job_test_real_df['index'] = pd.to_datetime(job_test_real_df['index'], format="%Y.%m.").dt.strftime("%Y-%m")
job_test_real_df.set_index('index').to_csv("../전처리/실업자_전처리(천명).csv", encoding="CP949")

# 밑에 파일 이름 바꾸는 것도 잊지마세요 -> 괜히 덮어쓰기 처리 되면 돌리는 사람만 억울함

In [10]:
# 코스피 지수 추출 (2004.1.1 ~ 2022.7.31)
import pandas_datareader as pdd
import datetime

start = datetime.datetime(2004,1,1)
end = datetime.datetime(2022,7,31)
df_null = pdd.DataReader("^KS11", "yahoo", start, end)
df = df_null.dropna()

df.to_csv("../전처리/kospi.csv", encoding="CP949")

In [11]:
# LTV랑 DTI 가져와서 처리하기
# 필요한 컬럼만 별도로 ltt_real로 빼서 컬럼 이름을 바꿔줍시다
ltt = pd.read_csv("../raw_data/LTV_DTI.csv", header=None)
ltt_real = ltt[[0,1,2,5,6,7]]
ltt_real = ltt_real.rename(columns={0:"발행회차", 1:"발행일자", 2:"기준월", 5:"평균 LTV", 6:"평균 DTI", 7:"조기상환율"}).reset_index(drop=True)
#ltt_real

In [12]:
# LTV랑 DTI도 시계열 처리하고 저장하기

ltt_real['발행일자'] = pd.to_datetime(ltt_real['발행일자'], format="%Y-%m-%d").dt.strftime("%Y-%m-%d")
ltt_real['기준월'] = pd.to_datetime(ltt_real['기준월'], format="%Y-%m").dt.strftime("%Y-%m")
ltt_real.to_csv("../전처리/LTV_DTI_전처리.csv", encoding="CP949")

In [13]:
# CD 91물 금리 가져와서 불필요 컬럼 제거

cd_91 = pd.read_excel("../raw_data/CD금리.xls")
cd_real = cd_91.iloc[1:7].transpose().reset_index(drop=True).dropna()
cd_real.drop(columns=[2,3,4,5], inplace=True)

In [14]:
# 컬럼 이름 변경 후 시계열 처리하고 내보내기
cd_real = cd_real.rename(columns={1:'기준월', 6:'CD금리'})
cd_real['기준월'] = pd.to_datetime(cd_real['기준월'], format="%Y%m월").dt.strftime("%Y-%m")
cd_real.to_csv("../전처리/CD금리_전처리.csv")

In [15]:
# 미국 금리 봅시다
america_rate = pd.read_csv("../raw_data/FEDFUNDS.csv")

# merge 편하게 하려고 컬럼명 통일을 위해 이름 변경 후 시계열 처리하고 내보내기
america_rate = america_rate.rename(columns={'DATE':'기준월', 'FEDFUNDS':'미국 실효금리'})
america_rate['기준월'] = pd.to_datetime(america_rate['기준월'], format="%Y-%m-%d").dt.strftime("%Y-%m")
america_rate.to_csv("../전처리/미국실효금리_전처리.csv")

# 여기서 부터는 MERGE

In [16]:
# 데이터 가져올 때 encoding="CP949"
# 만약 CP949 안 먹히면 "UTF8"

gdp = pd.read_csv("../전처리/GDP.csv", encoding="CP949")
mbs = pd.read_csv("../전처리/mbs_end.csv", encoding="CP949")
mbs_gum = pd.read_csv("../전처리/mbs_발행금리.csv", encoding="UTF8", header=None)
sobi = pd.read_csv("../전처리/소비자물가지수.csv", encoding="CP949")
lost_job = pd.read_csv("../전처리/실업자_전처리(단위_천명).csv", encoding="CP949")
sell_house = pd.read_csv("../전처리/주택종합_매매가격지수_전처리.csv", encoding="CP949")
hold_house = pd.read_csv("../전처리/주택종합_전세가격지수_전처리.csv", encoding="CP949")
ltv_dti = pd.read_csv("../전처리/LTV_DTI_전처리.csv", encoding="CP949")
cd_91 = pd.read_csv("../전처리/CD금리_전처리.csv", encoding="UTF8")
america_rate = pd.read_csv("../전처리/미국실효금리_전처리.csv", encoding="UTF8")

In [17]:
# 일단 주택지수끼리 합쳐봅시다

house = pd.merge(sell_house, hold_house, on='index', how='outer')
house.drop(columns=['Unnamed: 0_x', 'Unnamed: 0_y'], inplace=True)

In [18]:
# 소비자물가지수 시계열 처리
sobi = sobi.rename(columns={'Unnamed: 0':'index'})
sobi['index'] = pd.to_datetime(sobi['index'], format="%Y/%m").dt.strftime("%Y-%m")

In [19]:
# 주택 지수랑 소비자 지수 합치기
# complete로 모든 데이터를 담을 데이터프레임을 만들어서 합치기

complete = pd.merge(house, sobi, on='index', how='outer')

In [20]:
# 실업자를 15세 이상이랑 15-64만 추출

lost_job_2 = lost_job[['index','15세 이상','15-64']]
lost_job_2 = lost_job_2.rename(columns={'15세 이상':'15세 이상 실업자 수', '15-64':'15-64 실업자 수'})

# complete 데이터 프레임에 합치기
complete = pd.merge(complete, lost_job_2, on='index', how='outer')

In [21]:
# 중간 상황 점검 (불필요 시 주석처리)
complete

,index,주택종합 매매가격지수,주택종합 전세가격지수,소비자물가지수,15세 이상 실업자 수,15-64 실업자 수
0,1986-01,27.682,15.133,NaN,NaN,NaN
1,1986-02,27.682,15.350,NaN,NaN,NaN
2,1986-03,27.724,15.711,NaN,NaN,NaN
3,1986-04,27.516,16.036,NaN,NaN,NaN
4,1986-05,27.392,16.072,NaN,NaN,NaN
...,...,...,...,...,...,...
434,2022-03,100.306,100.337,106.06,873.0,798.0
435,2022-04,100.517,100.543,106.85,864.0,813.0
436,2022-05,100.768,100.814,107.56,889.0,830.0
437,2022-06,100.869,100.912,108.22,888.0,824.0


In [22]:
# mbs의 불필요한 컬럼들 제거
mbs.drop(columns=['Unnamed: 0'], inplace=True)

# 발행일자, 기준월에 대한 시계열 처리 수행
mbs['발행일자'] = pd.to_datetime(mbs['발행일자'], format="%Y-%m-%d").dt.strftime("%Y-%m-%d")
mbs['기준월'] = pd.to_datetime(mbs['기준월'], format="%Y-%m").dt.strftime("%Y-%m")

# 발행회차와 기준월을 기준으로 재정렬
mbs = mbs.sort_values(['발행회차','기준월'], ascending=True).reset_index(drop=True)
mbs.to_csv("../전처리/mbs_test.csv", encoding='CP949')

In [23]:
# mbs 금리 헤더 제거
mbs_gum2 = mbs_gum[1:].copy()

# 불필요 컬럼 제거
mbs_gum2.drop(columns=[3,4,5,6,7,8], inplace=True)

# 컬럼 이름 변경
mbs_gum2 = mbs_gum2.rename(columns={0:'발행회차', 1:'발행일자', 2:'가중평균발행금리'})

# 발행일자, 기준월에 대한 시계열 처리 수행
mbs_gum2['발행일자'] = pd.to_datetime(mbs_gum2['발행일자'], format="%Y-%m-%d").dt.strftime("%Y-%m-%d")
#mbs_gum2['기준월'] = pd.to_datetime(mbs_gum2['기준월'], format="%Y-%m").dt.strftime("%Y-%m")

# 발행회차와 기준월을 기준으로 재정렬
mbs_gum2 = mbs_gum2.sort_values(['발행회차'], ascending=True).reset_index(drop=True)
mbs_gum2.to_csv("../전처리/mbs_금리_최종.csv", encoding='CP949')
mbs_gum2

,발행회차,발행일자,가중평균발행금리
0,MBS 2004-01,2004-06-15,5.04
1,MBS 2004-02,2004-07-28,5
2,MBS 2004-03,2004-08-18,4.5
3,MBS 2004-04,2004-09-23,4.26
4,MBS 2004-05,2004-10-28,4.2
...,...,...,...
410,MBS 2022-12,2022-06-21,4.246
411,MBS 2022-13,2022-06-28,4.336
412,MBS 2022-14,2022-07-12,4.093
413,MBS 2022-15,2022-07-22,4.046


In [24]:
# 여기서 중간에 에러가 나오면 mbs_real 컬럼 확인해주세요
# MBS 발행 금리와 기존 MBS 데이터 merge
mbs_real = pd.merge(mbs, mbs_gum2, on='발행회차', how='outer')
mbs_real.drop(columns=['발행일자_x'], inplace=True)

In [25]:
# 발행회차와 기준월을 기준으로 다시 재정렬
mbs_real = mbs_real.sort_values(['발행회차', '기준월'], ascending=True).reset_index(drop=True).dropna()
mbs_real.to_csv("../전처리/mbs_merge_real.csv", encoding='CP949')

In [26]:
# MBS merge 잘 됐나 확인하기 (불필요 시 주석처리)
mbs_real

,발행회차,기준월,만기전상환율,발행일자_y,가중평균발행금리
0,MBS 2004-01,2004-06,0.5100,2004-06-15,5.04
1,MBS 2004-01,2004-07,0.3600,2004-06-15,5.04
2,MBS 2004-01,2004-08,0.4200,2004-06-15,5.04
3,MBS 2004-01,2004-09,0.2900,2004-06-15,5.04
4,MBS 2004-01,2004-10,0.4200,2004-06-15,5.04
...,...,...,...,...,...
34712,MBS 2022-10,2022-05,0.2421,2022-05-24,3.929
34713,MBS 2022-10,2022-06,0.3491,2022-05-24,3.929
34714,MBS 2022-11,2022-06,0.2994,2022-06-10,4.134
34715,MBS 2022-12,2022-06,0.2903,2022-06-21,4.246


In [27]:
# complete로 담아놨던 것들에서 결측치 제거
complete2 = complete.dropna().copy()
complete2

,index,주택종합 매매가격지수,주택종합 전세가격지수,소비자물가지수,15세 이상 실업자 수,15-64 실업자 수
216,2004-01,52.586,51.660,71.172,912.0,904.0
217,2004-02,52.693,51.773,71.487,955.0,946.0
218,2004-03,52.800,51.959,72.181,934.0,926.0
219,2004-04,52.888,51.994,72.181,865.0,856.0
220,2004-05,52.879,51.869,72.119,833.0,820.0
...,...,...,...,...,...,...
433,2022-02,100.207,100.197,105.300,954.0,800.0
434,2022-03,100.306,100.337,106.060,873.0,798.0
435,2022-04,100.517,100.543,106.850,864.0,813.0
436,2022-05,100.768,100.814,107.560,889.0,830.0


In [28]:
# 그동안 했던 것들 합쳐보기
all_data = pd.merge(complete2, mbs_real, left_on='index', right_on='기준월')
all_data

,index,주택종합 매매가격지수,주택종합 전세가격지수,소비자물가지수,15세 이상 실업자 수,15-64 실업자 수,발행회차,기준월,만기전상환율,발행일자_y,가중평균발행금리
0,2004-06,52.743,51.435,72.119,803.0,794.0,MBS 2004-01,2004-06,0.5100,2004-06-15,5.04
1,2004-07,52.603,51.040,72.561,867.0,858.0,MBS 2004-01,2004-07,0.3600,2004-06-15,5.04
2,2004-07,52.603,51.040,72.561,867.0,858.0,MBS 2004-02,2004-07,0.1800,2004-07-28,5
3,2004-08,52.394,50.554,73.191,849.0,842.0,MBS 2004-01,2004-08,0.4200,2004-06-15,5.04
4,2004-08,52.394,50.554,73.191,849.0,842.0,MBS 2004-02,2004-08,0.3900,2004-07-28,5
...,...,...,...,...,...,...,...,...,...,...,...
34712,2022-06,100.869,100.912,108.220,888.0,824.0,MBS 2022-09,2022-06,0.2224,2022-05-13,3.997
34713,2022-06,100.869,100.912,108.220,888.0,824.0,MBS 2022-10,2022-06,0.3491,2022-05-24,3.929
34714,2022-06,100.869,100.912,108.220,888.0,824.0,MBS 2022-11,2022-06,0.2994,2022-06-10,4.134
34715,2022-06,100.869,100.912,108.220,888.0,824.0,MBS 2022-12,2022-06,0.2903,2022-06-21,4.246


In [29]:
# kospi 일일 데이터를 가져와서 월별 데이터로 바꾸기
import FinanceDataReader as fdr

kospi_d = fdr.DataReader('KS11', '2004-01-01', '2022-07-31')
kospi_m = kospi_d.resample('M').last().reset_index()

# kospi 데이터 시계열 다시 처리
kospi_m['Date'] = pd.to_datetime(kospi_m['Date'], format="%Y-%m-%d").dt.strftime("%Y-%m")

In [30]:
# 필요한 컬럼 추출하기
kopsi_need = kospi_m[['Date','Close']]

# 다시 merge
all_data = pd.merge(all_data, kopsi_need, left_on='기준월', right_on='Date')

In [31]:
# 불필요한 컬럼 제거
all_data.drop(columns=['index', '15세 이상 실업자 수', 'Date'], inplace=True)

In [32]:
# 컬럼명 변경
all_data = all_data.rename(columns={'Close':'코스피 월별 종가'})

In [33]:
# 예금금리 추가
banking = pd.read_csv("../raw_data/(월별)예금은행 대출금리(신규취급액 기준)_200401-202206.csv")

# 예금금리 시계열 처리
banking['변환'] = pd.to_datetime(banking['변환'], format="%Y/%m").dt.strftime("%Y-%m")

In [34]:
# 다시 merge
all_data = pd.merge(all_data, banking, left_on='기준월', right_on='변환')

# 다시 불필요한 컬럼 제거
all_data.drop(columns=['변환'], inplace=True)

In [35]:
# 컬럼 재배치
all_data = all_data[['발행회차', '발행일자_y', '기준월', '만기전상환율', '가중평균발행금리', '주택종합 매매가격지수', '주택종합 전세가격지수',
'소비자물가지수', '15-64 실업자 수', '코스피 월별 종가', '원자료']]
all_data

,발행회차,발행일자_y,기준월,만기전상환율,가중평균발행금리,주택종합 매매가격지수,주택종합 전세가격지수,소비자물가지수,15-64 실업자 수,코스피 월별 종가,원자료
0,MBS 2004-01,2004-06-15,2004-06,0.5100,5.04,52.743,51.435,72.119,794.0,785.79,5.99
1,MBS 2004-01,2004-06-15,2004-07,0.3600,5.04,52.603,51.040,72.561,858.0,735.34,5.93
2,MBS 2004-02,2004-07-28,2004-07,0.1800,5,52.603,51.040,72.561,858.0,735.34,5.93
3,MBS 2004-01,2004-06-15,2004-08,0.4200,5.04,52.394,50.554,73.191,842.0,803.57,5.80
4,MBS 2004-02,2004-07-28,2004-08,0.3900,5,52.394,50.554,73.191,842.0,803.57,5.80
...,...,...,...,...,...,...,...,...,...,...,...
34712,MBS 2022-09,2022-05-13,2022-06,0.2224,3.997,100.869,100.912,108.220,824.0,2332.64,4.04
34713,MBS 2022-10,2022-05-24,2022-06,0.3491,3.929,100.869,100.912,108.220,824.0,2332.64,4.04
34714,MBS 2022-11,2022-06-10,2022-06,0.2994,4.134,100.869,100.912,108.220,824.0,2332.64,4.04
34715,MBS 2022-12,2022-06-21,2022-06,0.2903,4.246,100.869,100.912,108.220,824.0,2332.64,4.04


In [36]:
# 컬럼 이름바꾸기
all_data = all_data.rename(columns={'원자료':'예금은행 대출금리', '발행일자_y':'발행일자'})

# 확인 차원에서 데이터 내보내기
all_data.to_csv("../전처리/전처리_최종.csv", encoding='CP949')

In [37]:
# 중간점검 (불필요 시 주석처리)
all_data

,발행회차,발행일자,기준월,만기전상환율,가중평균발행금리,주택종합 매매가격지수,주택종합 전세가격지수,소비자물가지수,15-64 실업자 수,코스피 월별 종가,예금은행 대출금리
0,MBS 2004-01,2004-06-15,2004-06,0.5100,5.04,52.743,51.435,72.119,794.0,785.79,5.99
1,MBS 2004-01,2004-06-15,2004-07,0.3600,5.04,52.603,51.040,72.561,858.0,735.34,5.93
2,MBS 2004-02,2004-07-28,2004-07,0.1800,5,52.603,51.040,72.561,858.0,735.34,5.93
3,MBS 2004-01,2004-06-15,2004-08,0.4200,5.04,52.394,50.554,73.191,842.0,803.57,5.80
4,MBS 2004-02,2004-07-28,2004-08,0.3900,5,52.394,50.554,73.191,842.0,803.57,5.80
...,...,...,...,...,...,...,...,...,...,...,...
34712,MBS 2022-09,2022-05-13,2022-06,0.2224,3.997,100.869,100.912,108.220,824.0,2332.64,4.04
34713,MBS 2022-10,2022-05-24,2022-06,0.3491,3.929,100.869,100.912,108.220,824.0,2332.64,4.04
34714,MBS 2022-11,2022-06-10,2022-06,0.2994,4.134,100.869,100.912,108.220,824.0,2332.64,4.04
34715,MBS 2022-12,2022-06-21,2022-06,0.2903,4.246,100.869,100.912,108.220,824.0,2332.64,4.04


In [38]:
# LTV DTI 가져오면서 불필요한 컬럼 제거
ltv_dti.drop(columns=['Unnamed: 0', '발행회차'], inplace=True)

In [39]:
# 발행회차와 기준월을 기준으로 다시 재정렬
ltv_real = ltv_dti.sort_values(['발행일자', '기준월'], ascending=True).reset_index(drop=True).dropna()
ltv_real

,발행일자,기준월,평균 LTV,평균 DTI,조기상환율
0,2004-06-15,2004-06,60.66,34.53,0.5100
1,2004-06-15,2004-07,60.67,34.53,0.3600
2,2004-06-15,2004-08,60.67,34.53,0.4200
3,2004-06-15,2004-09,60.68,34.55,0.2900
4,2004-06-15,2004-10,60.69,34.53,0.4200
...,...,...,...,...,...
33172,2022-05-24,2022-05,59.81,29.34,0.2421
33173,2022-05-24,2022-06,59.84,29.35,0.3491
33174,2022-06-10,2022-06,60.47,29.01,0.2994
33175,2022-06-21,2022-06,64.02,32.01,0.2903


In [40]:
# Never Ending Merge
real_data = pd.merge(all_data, ltv_real, on=['발행일자', '기준월'], how="left")

# 다시 재정렬하고 결측치 없애고
real_data = real_data.sort_values(['발행일자', '기준월'], ascending=True).reset_index(drop=True).dropna()
#real_data.to_csv("../전처리/전처리_최종_V2.csv", encoding="CP949")

- 데이터가 어느 정도 merge 된 상태에서 다시 한 번 merge를 하려고 했더니
- ['발행일자'] 또는 ['발행월'] 단독으로 기준을 잡으면 데이터가 기하급수적으로 불어납니다
- ['발행일자'],['발행월'] merge on 기준을 두 개로 잡아서 과도한 merge를 방지합시다

In [41]:
# CD 91 금리 추가
real_data = pd.merge(real_data, cd_91, on='기준월', how='inner')
real_data.drop(columns=['Unnamed: 0'], inplace=True)

In [42]:
# 미국 실효금리(EFFR) 추가
real_data = pd.merge(real_data, america_rate, on='기준월', how='inner')
real_data.drop(columns=['Unnamed: 0'], inplace=True)

In [43]:
# 진짜 진짜 끝
real_data.to_csv("../전처리/전처리_최종_V3.csv", encoding="CP949")

In [44]:
# 확인
real_data

,발행회차,발행일자,기준월,만기전상환율,가중평균발행금리,주택종합 매매가격지수,주택종합 전세가격지수,소비자물가지수,15-64 실업자 수,코스피 월별 종가,예금은행 대출금리,평균 LTV,평균 DTI,조기상환율,CD금리,미국 실효금리
0,MBS 2004-01,2004-06-15,2004-06,0.5100,5.04,52.743,51.435,72.119,794.0,785.79,5.99,60.66,34.53,0.5100,3.91,1.03
1,MBS 2004-01,2004-06-15,2004-07,0.3600,5.04,52.603,51.040,72.561,858.0,735.34,5.93,60.67,34.53,0.3600,3.91,1.26
2,MBS 2004-02,2004-07-28,2004-07,0.1800,5,52.603,51.040,72.561,858.0,735.34,5.93,61.51,24.01,0.1800,3.91,1.26
3,MBS 2004-01,2004-06-15,2004-08,0.4200,5.04,52.394,50.554,73.191,842.0,803.57,5.80,60.67,34.53,0.4200,3.70,1.43
4,MBS 2004-02,2004-07-28,2004-08,0.3900,5,52.394,50.554,73.191,842.0,803.57,5.80,61.52,24.03,0.3900,3.70,1.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34661,MBS 2022-09,2022-05-13,2022-06,0.2224,3.997,100.869,100.912,108.220,824.0,2332.64,4.04,61.44,29.19,0.2224,1.99,1.21
34662,MBS 2022-10,2022-05-24,2022-06,0.3491,3.929,100.869,100.912,108.220,824.0,2332.64,4.04,59.84,29.35,0.3491,1.99,1.21
34663,MBS 2022-11,2022-06-10,2022-06,0.2994,4.134,100.869,100.912,108.220,824.0,2332.64,4.04,60.47,29.01,0.2994,1.99,1.21
34664,MBS 2022-12,2022-06-21,2022-06,0.2903,4.246,100.869,100.912,108.220,824.0,2332.64,4.04,64.02,32.01,0.2903,1.99,1.21


In [45]:
# merge_test.loc[merge_test.isnull()[],:]
# F열에서 nan값을 포함하는 행만 추출하기        ## 결측치가 있는 행의 위치를 뽑아줌 #중요
# merge_test.to_csv("../전처리/결측치_확인용.csv", encoding="CP949")